In [1]:
# ----------- imports

import pandas as pd 
import io
import requests
import time

import yfinance as yf  # https://github.com/ranaroussi/yfinance

In [41]:
# ----------- parameters

EXCHANGE_LIST = ['nyse', 'nasdaq', 'amex']  # from https://www.nasdaq.com/market-activity/stocks/screener
# EXCHANGE_LIST = ['nyse']
MARKET_CAP_MIN = 100000  # in millions

EXPORT_TO_CSV_FILE = 'data.csv'

# constants (do not change)
COUNTRY_USA = 'United States'
ONE_MILLION = 1000000

In [3]:
# ----------- support functions

# headers and params used to bypass NASDAQ's anti-scraping mechanism in function __exchange2df
nasdaq_headers = {
    'authority': 'api.nasdaq.com',
    'accept': 'application/json, text/plain, */*',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
    'origin': 'https://www.nasdaq.com',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.nasdaq.com/',
    'accept-language': 'en-US,en;q=0.9',
}

def nasdaq_params(exchange):
    return (
        ('letter', '0'),
        ('exchange', exchange),
        ('download', 'true'),
    )

def exchange_to_df(exchange):
    # ref: https://github.com/shilewenuw/get_all_tickers
    r = requests.get('https://api.nasdaq.com/api/screener/stocks', \
                     headers=nasdaq_headers, params=nasdaq_params(exchange))
    data = r.json()['data']
    df = pd.DataFrame(data['rows'], columns=data['headers'])
    df_filtered = df[~df['symbol'].str.contains("\.|\^")]  # remove weird tickers
    return df_filtered

def df_of_nyse_nasdaq_amex():
    df_all = pd.DataFrame()
    
    for exchange in EXCHANGE_LIST:
        start_time = time.time()

        df = exchange_to_df(exchange)
        df = df[df['country'] == COUNTRY_USA]  # only US companies
        df = df[df['marketCap'] != '']  # no empty market cap
        df = df[df['marketCap'].astype(float) >= MARKET_CAP_MIN * ONE_MILLION]  # filter by market cap
        
        df_all = df_all.append(df, ignore_index=True)
        
        print('  %s (US companies > %s millions): %.2f seconds' % \
              (exchange, MARKET_CAP_MIN, time.time() - start_time))
    return df_all

In [4]:
# ----------- pull data and store to dataframe (US companies with MARKET_CAP_MIN)

print('Exchange list:')
df_usa_mktcap = df_of_nyse_nasdaq_amex()

Exchange list:
  nyse (US companies > 100000 millions): 2.68 seconds
  nasdaq (US companies > 100000 millions): 5.23 seconds
  amex (US companies > 100000 millions): 1.25 seconds


In [5]:
# ----------- get only the tickers/symbols

tickers_usa_mktcap = df_usa_mktcap['symbol'].tolist()
# print('Total number of tickers: %s' % (len(tickers_usa_mktcap)))

# manually add DASH to the ticker list
tickers_usa_mktcap.append('DASH')
print('Total number of tickers: %s' % (len(tickers_usa_mktcap)))

Total number of tickers: 87


In [8]:
# ----------- check the data that yfinance support

# msft = yf.Ticker("MSFT")  # use Microsoft as an example

In [ ]:
# msft.info  # equivalent to https://in.finance.yahoo.com/quote/MSFT/key-statistics?p=MSFT

# Output:

# {'zip': '98052-6399',
#  'sector': 'Technology',
#  'fullTimeEmployees': 163000,
#  'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, organizations, and enterprise divisions. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and desktop solutions; and training and certification to developers and IT professionals on Microsoft products. Its More Personal Computing segment provides Windows original equipment manufacturer (OEM) licensing and other non-volume licensing of the Windows operating system; Windows Commercial, such as volume licensing of the Windows operating system, Windows cloud services, and other Windows commercial offerings; patent licensing; Windows Internet of Things; and MSN advertising. It also offers Surface, PC accessories, PCs, tablets, gaming and entertainment consoles, and other devices; Gaming, including Xbox hardware, and Xbox content and services; video games and third-party video game royalties; and Search, including Bing and Microsoft advertising. It sells its products through OEMs, distributors, and resellers; and directly through digital marketplaces, online stores, and retail stores. It has a collaboration with DXC Technology, Dynatrace, Inc., Morgan Stanley, Micro Focus, WPP plc, and iCIMS, Inc. The company was founded in 1975 and is headquartered in Redmond, Washington.',
#  'city': 'Redmond',
#  'phone': '425 882 8080',
#  'state': 'WA',
#  'country': 'United States',
#  'companyOfficers': [],
#  'website': 'http://www.microsoft.com',
#  'maxAge': 1,
#  'address1': 'One Microsoft Way',
#  'industry': 'Software—Infrastructure',
#  'previousClose': 266.69,
#  'regularMarketOpen': 266.23,
#  'twoHundredDayAverage': 240.0639,
#  'trailingAnnualDividendYield': 0.008211781,
#  'payoutRatio': 0.2916,
#  'volume24Hr': None,
#  'regularMarketDayHigh': 267.25,
#  'navPrice': None,
#  'averageDailyVolume10Day': 25841766,
#  'totalAssets': None,
#  'regularMarketPreviousClose': 266.69,
#  'fiftyDayAverage': 252.55824,
#  'trailingAnnualDividendRate': 2.19,
#  'open': 266.23,
#  'toCurrency': None,
#  'averageVolume10days': 25841766,
#  'expireDate': None,
#  'yield': None,
#  'algorithm': None,
#  'dividendRate': 2.24,
#  'exDividendDate': 1629244800,
#  'beta': 0.78913,
#  'circulatingSupply': None,
#  'startDate': None,
#  'regularMarketDayLow': 264.76,
#  'priceHint': 2,
#  'currency': 'USD',
#  'trailingPE': 36.116108,
#  'regularMarketVolume': 25611113,
#  'lastMarket': None,
#  'maxSupply': None,
#  'openInterest': None,
#  'marketCap': 1996016582656,
#  'volumeAllCurrencies': None,
#  'strikePrice': None,
#  'averageVolume': 25470776,
#  'priceToSalesTrailing12Months': 12.477521,
#  'dayLow': 264.76,
#  'ask': 265.17,
#  'ytdReturn': None,
#  'askSize': 1300,
#  'volume': 25611113,
#  'fiftyTwoWeekHigh': 267.85,
#  'forwardPE': 31.663082,
#  'fromCurrency': None,
#  'fiveYearAvgDividendYield': 1.6,
#  'fiftyTwoWeekLow': 193.55,
#  'bid': 265.02,
#  'tradeable': False,
#  'dividendYield': 0.0084,
#  'bidSize': 800,
#  'dayHigh': 267.25,
#  'exchange': 'NMS',
#  'shortName': 'Microsoft Corporation',
#  'longName': 'Microsoft Corporation',
#  'exchangeTimezoneName': 'America/New_York',
#  'exchangeTimezoneShortName': 'EDT',
#  'isEsgPopulated': False,
#  'gmtOffSetMilliseconds': '-14400000',
#  'quoteType': 'EQUITY',
#  'symbol': 'MSFT',
#  'messageBoardId': 'finmb_21835',
#  'market': 'us_market',
#  'annualHoldingsTurnover': None,
#  'enterpriseToRevenue': 12.283,
#  'beta3Year': None,
#  'profitMargins': 0.35016,
#  'enterpriseToEbitda': 25.998,
#  '52WeekChange': 0.33551705,
#  'morningStarRiskRating': None,
#  'forwardEps': 8.37,
#  'revenueQuarterlyGrowth': None,
#  'sharesOutstanding': 7531570176,
#  'fundInceptionDate': None,
#  'annualReportExpenseRatio': None,
#  'bookValue': 17.853,
#  'sharesShort': 48137624,
#  'sharesPercentSharesOut': 0.0064,
#  'fundFamily': None,
#  'lastFiscalYearEnd': 1593475200,
#  'heldPercentInstitutions': 0.72173,
#  'netIncomeToCommon': 56014999552,
#  'trailingEps': 7.338,
#  'lastDividendValue': 0.56,
#  'SandP52WeekChange': 0.4020188,
#  'priceToBook': 14.844563,
#  'heldPercentInsiders': 0.00071999995,
#  'nextFiscalYearEnd': 1656547200,
#  'mostRecentQuarter': 1617148800,
#  'shortRatio': 2.24,
#  'sharesShortPreviousMonthDate': 1620950400,
#  'floatShares': 7420408511,
#  'enterpriseValue': 1964842680320,
#  'threeYearAverageReturn': None,
#  'lastSplitDate': 1045526400,
#  'lastSplitFactor': '2:1',
#  'legalType': None,
#  'lastDividendDate': 1621382400,
#  'morningStarOverallRating': None,
#  'earningsQuarterlyGrowth': 0.438,
#  'dateShortInterest': 1623715200,
#  'pegRatio': 1.98,
#  'lastCapGain': None,
#  'shortPercentOfFloat': 0.0064,
#  'sharesShortPriorMonth': 40526093,
#  'impliedSharesOutstanding': None,
#  'category': None,
#  'fiveYearAverageReturn': None,
#  'regularMarketPrice': 265.02,
#  'logo_url': 'https://logo.clearbit.com/microsoft.com'}

In [7]:
# print(msft.history(period="max"))

# Output:

#                  Open        High         Low       Close      Volume    Dividends  Stock Splits
# Date                                                                     
# 1986-03-13    0.056108    0.064359    0.056108    0.061608  1031788800         0.0           0.0
# 1986-03-14    0.061608    0.064909    0.061608    0.063809   308160000         0.0           0.0
# 1986-03-17    0.063809    0.065459    0.063809    0.064909   133171200         0.0           0.0
# 1986-03-18    0.064909    0.065459    0.062708    0.063258    67766400         0.0           0.0
# 1986-03-19    0.063258    0.063809    0.061608    0.062158    47894400         0.0           0.0
# ...                ...         ...         ...         ...         ...   
# 2021-06-21  259.820007  263.519989  257.920013  262.630005    26696100         0.0           0.0
# 2021-06-22  262.720001  265.790009  262.399994  265.510010    24694100         0.0           0.0
# 2021-06-23  265.989990  266.829987  264.429993  265.269989    19518700         0.0           0.0
# 2021-06-24  266.160004  267.850006  265.470001  266.690002    21446900         0.0           0.0
# 2021-06-25  266.230011  267.250000  264.760010  265.019989    25604200         0.0           0.0
# [8895 rows x 7 columns]

In [9]:
# print(msft.actions)

# Output:

#             Dividends  Stock Splits
# Date                               
# 1987-09-21       0.00           2.0
# 1990-04-16       0.00           2.0
# 1991-06-27       0.00           1.5
# 1992-06-15       0.00           1.5
# 1994-05-23       0.00           2.0
# ...               ...           ...
# 2020-05-20       0.51           0.0
# 2020-08-19       0.51           0.0
# 2020-11-18       0.56           0.0
# 2021-02-17       0.56           0.0
# 2021-05-19       0.56           0.0
# [79 rows x 2 columns]

In [10]:
# msft.dividends

# Output:

# Date
# 2003-02-19    0.08
# 2003-10-15    0.16
# 2004-08-23    0.08
# 2004-11-15    3.08
# 2005-02-15    0.08
#               ... 
# 2020-05-20    0.51
# 2020-08-19    0.51
# 2020-11-18    0.56
# 2021-02-17    0.56
# 2021-05-19    0.56
# Name: Dividends, Length: 70, dtype: float64

In [11]:
# msft.splits

# Output:

# Date
# 1987-09-21    2.0
# 1990-04-16    2.0
# 1991-06-27    1.5
# 1992-06-15    1.5
# 1994-05-23    2.0
# 1996-12-09    2.0
# 1998-02-23    2.0
# 1999-03-29    2.0
# 2003-02-18    2.0
# Name: Stock Splits, dtype: float64

In [12]:
# print(msft.financials)  # equivalent to https://in.finance.yahoo.com/quote/MSFT/financials?p=MSFT

# Output:

#                                             2020-06-30      2019-06-30      2018-06-30     2017-06-30
# Research Development                     19269000000.0   16876000000.0   14726000000.0  13037000000.0
# Effect Of Accounting Charges                      None            None            None           None
# Income Before Tax                        53036000000.0   43688000000.0   36474000000.0  29901000000.0
# Minority Interest                                 None            None            None           None
# Net Income                               44281000000.0   39240000000.0   16571000000.0  25489000000.0
# Selling General Administrative           24709000000.0   23098000000.0   22223000000.0  19942000000.0
# Gross Profit                             96937000000.0   82933000000.0   72007000000.0  62310000000.0
# Ebit                                     52959000000.0   42959000000.0   35058000000.0  29331000000.0
# Operating Income                         52959000000.0   42959000000.0   35058000000.0  29331000000.0
# Other Operating Expenses                          None            None            None           None
# Interest Expense                         -2591000000.0   -2686000000.0   -2733000000.0  -2222000000.0
# Extraordinary Items                               None            None            None           None
# Non Recurring                                     None            None            None           None
# Other Items                                       None            None            None           None
# Income Tax Expense                        8755000000.0    4448000000.0   19903000000.0   4412000000.0
# Total Revenue                           143015000000.0  125843000000.0  110360000000.0  96571000000.0
# Total Operating Expenses                 90056000000.0   82884000000.0   75302000000.0  67240000000.0
# Cost Of Revenue                          46078000000.0   42910000000.0   38353000000.0  34261000000.0
# Total Other Income Expense Net              77000000.0     729000000.0    1416000000.0    570000000.0
# Discontinued Operations                           None            None            None           None
# Net Income From Continuing Ops           44281000000.0   39240000000.0   16571000000.0  25489000000.0
# Net Income Applicable To Common Shares   44281000000.0   39240000000.0   16571000000.0  25489000000.0

In [13]:
# print(msft.quarterly_financials)

# Output:

#                                            2021-03-31     2020-12-31      2020-09-30     2020-06-30
# Research Development                     5204000000.0   4899000000.0    4926000000.0   5214000000.0
# Effect Of Accounting Charges                     None           None            None           None
# Income Before Tax                       17236000000.0  18337000000.0   16124000000.0  13422000000.0
# Minority Interest                                None           None            None           None
# Net Income                              15457000000.0  15463000000.0   13893000000.0  11202000000.0
# Selling General Administrative           6409000000.0   6086000000.0    5350000000.0   7073000000.0
# Gross Profit                            28661000000.0  28882000000.0   26152000000.0  25694000000.0
# Ebit                                    17048000000.0  17897000000.0   15876000000.0  13407000000.0
# Operating Income                        17048000000.0  17897000000.0   15876000000.0  13407000000.0
# Other Operating Expenses                         None           None            None           None
# Interest Expense                         -628000000.0   -568000000.0    -585000000.0   -684000000.0
# Extraordinary Items                              None           None            None           None
# Non Recurring                                    None           None            None           None
# Other Items                                      None           None            None           None
# Income Tax Expense                       1779000000.0   2874000000.0    2231000000.0   2220000000.0
# Total Revenue                           41706000000.0  43076000000.0   37154000000.0  38033000000.0
# Total Operating Expenses                24658000000.0  25179000000.0   21278000000.0  24626000000.0
# Cost Of Revenue                         13045000000.0  14194000000.0   11002000000.0  12339000000.0
# Total Other Income Expense Net            188000000.0    440000000.0     248000000.0     15000000.0
# Discontinued Operations                          None           None            None           None
# Net Income From Continuing Ops          15457000000.0  15463000000.0   13893000000.0  11202000000.0
# Net Income Applicable To Common Shares  15457000000.0  15463000000.0   13893000000.0  11202000000.0

In [14]:
# print(msft.major_holders)

# Output:

#         0                                      1
# 0   0.07%        % of Shares Held by All Insider
# 1  72.17%       % of Shares Held by Institutions
# 2  72.22%        % of Float Held by Institutions
# 3    5329  Number of Institutions Holding Shares

In [15]:
# print(msft.institutional_holders)

# Output:

#                               Holder     Shares Date Reported   % Out          Value
# 0         Vanguard Group, Inc. (The)  610849992    2021-03-30  0.0811   144020102613
# 1                     Blackrock Inc.  518649291    2021-03-30  0.0689   122281943339
# 2           State Street Corporation  292575655    2021-03-30  0.0388    68980562179
# 3                           FMR, LLC  217585928    2021-03-30  0.0289    51300234244
# 4      Price (T.Rowe) Associates Inc  178824616    2021-03-30  0.0237    42161479714
# 5      Geode Capital Management, LLC  120231394    2021-03-30  0.0160    28346955763
# 6            Capital World Investors  112150578    2021-03-30  0.0149    26441741775
# 7  Capital Research Global Investors   99286940    2021-03-30  0.0132    23408881843
# 8    Capital International Investors   90228967    2021-03-30  0.0120    21273283549
# 9         Northern Trust Corporation   89762724    2021-03-30  0.0119    21163357437

In [16]:
# print(msft.balance_sheet)  # equivalent to https://in.finance.yahoo.com/quote/MSFT/balance-sheet?p=MSFT

# Output:

#                                     2020-06-30    2019-06-30    2018-06-30     2017-06-30
# Intangible Assets                 7.038000e+09  7.750000e+09  8.053000e+09   1.010600e+10
# Total Liab                        1.830070e+11  1.842260e+11  1.761300e+11   1.626010e+11
# Total Stockholder Equity          1.183040e+11  1.023300e+11  8.271800e+10   8.771100e+10
# Other Current Liab                4.600100e+10  4.586000e+10  3.819500e+10   3.087900e+10
# Total Assets                      3.013110e+11  2.865560e+11  2.588480e+11   2.503120e+11
# Common Stock                      8.055200e+10  7.852000e+10  7.122300e+10   6.931500e+10
# Other Current Assets              1.151700e+10  1.013300e+10  6.855000e+09   5.183000e+09
# Retained Earnings                 3.456600e+10  2.415000e+10  1.368200e+10   1.776900e+10
# Other Liab                        3.449200e+10  3.569900e+10  3.570700e+10   2.298600e+10
# Good Will                         4.335100e+10  4.202600e+10  3.568300e+10   3.512200e+10
# Treasury Stock                    3.186000e+09 -3.400000e+08 -2.187000e+09   6.270000e+08
# Other Assets                      1.313800e+10  1.472300e+10  7.442000e+09   6.076000e+09
# Cash                              1.357600e+10  1.135600e+10  1.194600e+10   7.663000e+09
# Total Current Liabilities         7.231000e+10  6.942000e+10  5.848800e+10   5.574500e+10
# Deferred Long Term Asset Charges  6.405000e+09  7.536000e+09  1.369000e+09   2.480000e+08
# Short Long Term Debt              3.749000e+09  5.516000e+09  3.998000e+09   1.049000e+09
# Other Stockholder Equity          3.186000e+09 -3.400000e+08 -2.187000e+09   6.270000e+08
# Property Plant Equipment          5.290400e+10  4.385600e+10  3.614600e+10   3.028900e+10
# Total Current Assets              1.819150e+11  1.755520e+11  1.696620e+11   1.626960e+11
# Long Term Investments             2.965000e+09  2.649000e+09  1.862000e+09   6.023000e+09
# Net Tangible Assets               6.791500e+10  5.255400e+10  3.898200e+10   4.248300e+10
# Short Term Investments            1.229160e+11  1.224760e+11  1.217180e+11   1.252380e+11
# Net Receivables                   3.201100e+10  2.952400e+10  2.648100e+10   2.243100e+10
# Long Term Debt                    5.957800e+10  6.666200e+10  7.224200e+10   7.607300e+10
# Inventory                         1.895000e+09  2.063000e+09  2.662000e+09   2.181000e+09
# Accounts Payable                  1.253000e+10  9.382000e+09  8.617000e+09   7.390000e+09       

In [17]:
# print(msft.quarterly_balance_sheet)

# Output:

#                                     2021-03-31    2020-12-31    2020-09-30     2020-06-30
# Intangible Assets                 8.127000e+09  6.555000e+09  6.923000e+09   7.038000e+09
# Total Liab                        1.743740e+11  1.739010e+11  1.776090e+11   1.830070e+11
# Total Stockholder Equity          1.345050e+11  1.302360e+11  1.233920e+11   1.183040e+11
# Other Current Liab                4.006900e+10  3.992600e+10  4.299600e+10   4.600100e+10
# Total Assets                      3.088790e+11  3.041370e+11  3.010010e+11   3.013110e+11
# Common Stock                      8.230800e+10  8.189600e+10  8.108900e+10   8.055200e+10
# Other Current Assets              1.203400e+10  1.274800e+10  1.357200e+10   1.151700e+10
# Retained Earnings                 5.073500e+10  4.497300e+10  3.919300e+10   3.456600e+10
# Other Liab                        3.160100e+10  3.158500e+10  3.317400e+10   3.449200e+10
# Good Will                         4.969800e+10  4.421900e+10  4.389000e+10   4.335100e+10
# Treasury Stock                    1.462000e+09  3.367000e+09  3.110000e+09   3.186000e+09
# Other Assets                      1.442700e+10  1.356100e+10  1.303400e+10   1.313800e+10
# Cash                              1.370200e+10  1.443200e+10  1.720500e+10   1.357600e+10
# Total Current Liabilities         7.219300e+10  6.748600e+10  7.005600e+10   7.231000e+10
# Short Long Term Debt              8.051000e+09  5.387000e+09  6.497000e+09   3.749000e+09
# Other Stockholder Equity          1.462000e+09  3.367000e+09  3.110000e+09   3.186000e+09
# Property Plant Equipment          6.561800e+10  6.203500e+10  5.697400e+10   5.290400e+10
# Total Current Assets              1.656140e+11  1.739730e+11  1.770770e+11   1.819150e+11
# Long Term Investments             5.395000e+09  3.794000e+09  3.103000e+09   2.965000e+09
# Net Tangible Assets               7.668000e+10  7.946200e+10  7.257900e+10   6.791500e+10
# Short Term Investments            1.113110e+11  1.175570e+11  1.207440e+11   1.229160e+11
# Net Receivables                   2.632200e+10  2.731200e+10  2.285100e+10   3.201100e+10
# Long Term Debt                    5.000700e+10  5.513600e+10  5.705500e+10   5.957800e+10 
# Inventory                         2.245000e+09  1.924000e+09  2.705000e+09   1.895000e+09
# Accounts Payable                  1.341200e+10  1.277000e+10  1.250900e+10   1.253000e+10
# Deferred Long Term Asset Charges           NaN           NaN           NaN   6.405000e+09

In [18]:
# print(msft.cashflow)  # equivalent to https://in.finance.yahoo.com/quote/MSFT/cash-flow?p=MSFT

# Output:

#                                              2020-06-30    2019-06-30      2018-06-30    2017-06-30
# Investments                                6.980000e+09  5.400000e+08    6.557000e+09 -1.251100e+10
# Change To Liabilities                      5.230000e+09  4.694000e+09    7.070000e+09  3.901000e+09
# Total Cashflows From Investing Activities -1.222300e+10 -1.577300e+10   -6.061000e+09 -4.678100e+10
# Net Borrowings                            -5.518000e+09 -4.000000e+09   -1.020100e+10  3.145900e+10
# Total Cash From Financing Activities      -4.603100e+10 -3.688700e+10   -3.359000e+10  8.408000e+09
# Change To Operating Activities            -6.730000e+08 -1.542000e+09   -4.590000e+08  3.490000e+08
# Issuance Of Stock                          1.343000e+09  1.142000e+09    1.002000e+09  7.720000e+08
# Net Income                                 4.428100e+10  3.924000e+10    1.657100e+10  2.548900e+10
# Change In Cash                             2.220000e+09 -5.900000e+08    4.283000e+09  1.153000e+09
# Repurchase Of Stock                       -2.296800e+10 -1.954300e+10   -1.072100e+10 -1.178800e+10
# Effect Of Exchange Rate                   -2.010000e+08 -1.150000e+08    5.000000e+07  1.900000e+07
# Total Cash From Operating Activities       6.067500e+10  5.218500e+10    4.388400e+10  3.950700e+10
# Depreciation                               1.230000e+10  1.160000e+10    9.900000e+09  7.800000e+09
# Other Cashflows From Investing Activities -1.241000e+09 -1.241000e+09   -9.800000e+07 -1.970000e+08
# Dividends Paid                            -1.513700e+10 -1.381100e+10   -1.269900e+10 -1.184500e+10
# Change To Inventory                        1.680000e+08  5.970000e+08   -4.650000e+08  5.000000e+07
# Change To Account Receivables             -2.577000e+09 -2.812000e+09   -3.862000e+09 -1.216000e+09
# Other Cashflows From Financing Activities -3.751000e+09 -6.750000e+08   -9.710000e+08 -1.900000e+08
# Change To Netincome                        5.577000e+09 -2.521000e+09   -3.054000e+09  1.342000e+09
# Capital Expenditures                      -1.544100e+10 -1.392500e+10   -1.163200e+10 -8.129000e+09         

In [19]:
# print(msft.quarterly_cashflow)

# Output:

#                                              2021-03-31    2020-12-31      2020-09-30    2020-06-30
# Investments                                2.517000e+09  2.593000e+09    2.100000e+09  3.173000e+09
# Change To Liabilities                      3.600000e+08 -3.194000e+09   -2.749000e+09  1.180200e+10
# Total Cashflows From Investing Activities -9.684000e+09 -1.669000e+09   -5.371000e+09 -4.462000e+09
# Net Borrowings                            -5.000000e+08 -3.250000e+09   -3.250000e+09 -3.250000e+09
# Total Cash From Financing Activities      -1.319200e+10 -1.363400e+10   -1.028900e+10 -1.226200e+10
# Change To Operating Activities             1.305000e+09  1.879000e+09   -2.823000e+09  6.430000e+08
# Issuance Of Stock                          3.960000e+08  3.020000e+08    5.450000e+08  3.400000e+08
# Net Income                                 1.545700e+10  1.546300e+10    1.389300e+10  1.120200e+10
# Change In Cash                            -7.300000e+08 -2.773000e+09    3.629000e+09  1.866000e+09
# Repurchase Of Stock                       -6.930000e+09 -6.535000e+09   -6.743000e+09 -5.791000e+09
# Effect Of Exchange Rate                   -3.300000e+07  1.400000e+07   -4.600000e+07 -8.300000e+07
# Total Cash From Operating Activities       2.217900e+10  1.251600e+10    1.933500e+10  1.867300e+10
# Depreciation                               2.936000e+09  2.761000e+09    2.645000e+09  3.008000e+09
# Other Cashflows From Investing Activities  4.000000e+08  3.270000e+08   -2.083000e+09 -1.241000e+09
# Dividends Paid                            -4.221000e+09 -4.230000e+09   -3.856000e+09 -3.865000e+09
# Change To Inventory                       -3.290000e+08  7.880000e+08   -8.080000e+08 -2.510000e+08
# Change To Account Receivables              2.900000e+08 -4.008000e+09    8.843000e+09 -9.355000e+09
# Other Cashflows From Financing Activities -1.937000e+09  7.900000e+07   -2.350000e+08 -2.946000e+09
# Change To Netincome                        1.086000e+09  1.195000e+09    1.317000e+09  2.213000e+09
# Capital Expenditures                      -5.089000e+09 -4.174000e+09   -4.907000e+09 -4.744000e+09                     

In [20]:
# print(msft.earnings)

# Output:

#            Revenue     Earnings
# Year                           
# 2017   96571000000  25489000000
# 2018  110360000000  16571000000
# 2019  125843000000  39240000000
# 2020  143015000000  44281000000

In [ ]:
# print(msft.quarterly_earnings)

# Output:

#              Revenue     Earnings
# Quarter                          
# 2Q2020   38033000000  11202000000
# 3Q2020   37154000000  13893000000
# 4Q2020   43076000000  15463000000
# 1Q2021   41706000000  15457000000

In [ ]:
# print(msft.sustainability)

# Output:

#                                      Value
# 2021-5                                    
# palmOil                              False
# controversialWeapons                 False
# gambling                             False
# socialScore                           9.37
# nuclear                              False
# furLeather                           False
# alcoholic                            False
# gmo                                  False
# catholic                             False
# socialPercentile                      None
# peerCount                              103
# governanceScore                       4.83
# environmentPercentile                 None
# animalTesting                        False
# tobacco                              False
# totalEsg                             14.63
# highestControversy                       3
# esgPerformance                  UNDER_PERF
# coal                                 False
# pesticides                           False
# adult                                False
# percentile                            7.59
# peerGroup              Software & Services
# smallArms                            False
# environmentScore                      0.42
# governancePercentile                  None
# militaryContract                     False

In [ ]:
# print(msft.recommendations)

# Output:

#                                Firm       To Grade From Grade Action
# Date                                                                
# 2012-03-16 08:19:00  Argus Research            Buy                up
# 2012-03-19 14:00:00  Hilliard Lyons  Long-Term Buy              main
# 2012-03-22 07:03:00  Morgan Stanley     Overweight              main
# 2012-04-03 11:53:00             UBS            Buy              main
# 2012-04-20 06:18:00   Deutsche Bank            Buy              main
# ...                             ...            ...        ...    ...
# 2021-04-28 13:18:05  Morgan Stanley     Overweight              main
# 2021-05-13 11:05:37      Rosenblatt            Buy              init
# 2021-05-18 13:39:15       Jefferies            Buy              main
# 2021-06-02 12:20:01  KGI Securities     Outperform              init
# 2021-06-23 11:22:46         Wedbush     Outperform              main
# [330 rows x 4 columns]

In [21]:
# print(msft.calendar)

# Output:

#                                     0                    1
# Earnings Date     2021-07-20 00:00:00  2021-07-26 00:00:00
# Earnings Average                  1.9                  1.9
# Earnings Low                     1.64                 1.64
# Earnings High                    2.03                 2.03
# Revenue Average           44102900000          44102900000
# Revenue Low               40850000000          40850000000
# Revenue High              44914700000          44914700000

In [ ]:
# msft.isin  # ISIN (International Securities Identification Number) code - *experimental*

# Output:

# 'US5949181045'

In [ ]:
# print(msft.options)  # options expirations
# opt = msft.option_chain('YYYY-MM-DD')  # option chain for specific expiration, available via: opt.calls, opt.puts

# Output:

# ('2021-07-02', '2021-07-09', '2021-07-16', '2021-07-23', '2021-07-30', '2021-08-06', '2021-08-20', 
#  '2021-09-17', '2021-10-15', '2021-11-19', '2022-01-21', '2022-03-18', '2022-06-17', '2022-09-16', 
#  '2023-01-20', '2023-03-17', '2023-06-16')

In [45]:
# ----------- get the data and store to a dataframe

LATEST_DATA_IDX = 0  # the first column of dataframe

start_time_all = time.time()
df_full = pd.DataFrame()

# for ticker in tickers_usa_mktcap:
for ticker in tickers_usa_mktcap[0:10]:
# for ticker in tickers_usa_mktcap[-1:]:  # get 'DASH'
    company = yf.Ticker(ticker)
#     company = yf.Ticker('MSFT')

    start_time = time.time()
    print('Getting data for %s... ' % ticker, end="")
    
    # {'zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'state', 'country',
    #  'companyOfficers', 'website', 'maxAge', 'address1', 'industry', 'previousClose', 'regularMarketOpen',
    #  'twoHundredDayAverage', 'trailingAnnualDividendYield', 'payoutRatio', 'volume24Hr', 'regularMarketDayHigh',
    #  'navPrice', 'averageDailyVolume10Day', 'totalAssets', 'regularMarketPreviousClose', 'fiftyDayAverage',
    #  'trailingAnnualDividendRate', 'open', 'toCurrency', 'averageVolume10days', 'expireDate', 'yield', 
    #  'algorithm', 'dividendRate', 'exDividendDate', 'beta', 'circulatingSupply', 'startDate', 
    #  'regularMarketDayLow', 'priceHint', 'currency', 'trailingPE', 'regularMarketVolume', 'lastMarket', 
    #  'maxSupply', 'openInterest', 'marketCap', 'volumeAllCurrencies', 'strikePrice', 'averageVolume', 
    #  'priceToSalesTrailing12Months', 'dayLow', 'ask', 'ytdReturn', 'askSize', 'volume', 'fiftyTwoWeekHigh', 
    #  'forwardPE', 'fromCurrency', 'fiveYearAvgDividendYield', 'fiftyTwoWeekLow', 'bid', 'tradeable', 
    #  'dividendYield', 'bidSize', 'dayHigh', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 
    #  'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 
    #  'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'profitMargins', 
    #  'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'forwardEps', 'revenueQuarterlyGrowth', 
    #  'sharesOutstanding', 'fundInceptionDate', 'annualReportExpenseRatio', 'bookValue', 'sharesShort', 
    #  'sharesPercentSharesOut', 'fundFamily', 'lastFiscalYearEnd', 'heldPercentInstitutions', 'netIncomeToCommon', 
    #  'trailingEps', 'lastDividendValue', 'SandP52WeekChange', 'priceToBook', 'heldPercentInsiders', 
    #  'nextFiscalYearEnd', 'mostRecentQuarter', 'shortRatio', 'sharesShortPreviousMonthDate', 'floatShares', 
    #  'enterpriseValue', 'threeYearAverageReturn', 'lastSplitDate', 'lastSplitFactor', 'legalType', 
    #  'lastDividendDate', 'morningStarOverallRating', 'earningsQuarterlyGrowth', 'dateShortInterest', 'pegRatio', 
    #  'lastCapGain', 'shortPercentOfFloat', 'sharesShortPriorMonth', 'impliedSharesOutstanding', 'category', 
    #  'fiveYearAverageReturn', 'regularMarketPrice', 'logo_url'}
    info = company.info  # is a map
    
    # [Research Development, Effect Of Accounting Charges, Income Before Tax, Minority Interest, Net Income, 
    #  Selling General Administrative, Gross Profit, Ebit, Operating Income, Other Operating Expenses, 
    #  Interest Expense, Extraordinary Items, Non Recurring, Other Items, Income Tax Expense, Total Revenue,
    #  Total Operating Expenses, Cost Of Revenue, Total Other Income Expense Net, Discontinued Operations,
    #  Net Income From Continuing Ops, Net Income Applicable To Common Shares]
    financials = company.financials.iloc[:, LATEST_DATA_IDX]  # company.financials is a dataframe

    # [Intangible Assets, Total Liab, Total Stockholder Equity, Other Current Liab, Total Assets, Common Stock,
    #  Other Current Assets, Retained Earnings, Other Liab, Good Will, Treasury Stock, Other Assets, Cash,
    #  Total Current Liabilities, Deferred Long Term Asset Charges, Short Long Term Debt, Other Stockholder Equity,
    #  Property Plant Equipment, Total Current Assets, Long Term Investments, Net Tangible Assets, 
    #  Short Term Investments, Net Receivables, Long Term Debt, Inventory, Accounts Payable]
    balance_sheet = company.balance_sheet.iloc[:, LATEST_DATA_IDX]  # company.balance_sheet is a dataframe
    
    # [Investments, Change To Liabilities, Total Cashflows From Investing Activities, Net Borrowings,
    #  Total Cash From Financing Activities, Change To Operating Activities, Issuance Of Stock, Net Income,
    #  Change In Cash, Repurchase Of Stock, Effect Of Exchange Rate, Total Cash From Operating Activities,
    #  Depreciation, Other Cashflows From Investing Activities, Dividends Paid, Change To Inventory,
    #  Change To Account Receivables, Other Cashflows From Financing Activities, Change To Netincome,
    #  Capital Expenditures]
    cash_flow = company.cashflow.iloc[:, LATEST_DATA_IDX]  # company.cashflow is a dataframe
    
    print('Done in %.2f seconds!' % (time.time() - start_time))
    
    # ----- debug -----
    
    # check for "assets"(s)
#     print('info[totalAssets]: %s' % (info['totalAssets']))

#     print('balance_sheet[Total Assets]                    : %s millions' % \
#           (balance_sheet['Total Assets'] / ONE_MILLION))
#     print('balance_sheet[Total Current Assets]            : %s millions' % \
#           (balance_sheet['Total Current Assets'] / ONE_MILLION))

#     print('balance_sheet[Other Assets]                    : %s millions' % \
#           (balance_sheet['Other Assets'] / ONE_MILLION))
#     print('balance_sheet[Other Current Assets]            : %s millions' % \
#           (balance_sheet['Other Current Assets'] / ONE_MILLION))

#     print('balance_sheet[Net Tangible Assets]             : %s millions' % \
#           (balance_sheet['Net Tangible Assets'] / ONE_MILLION))
#     print('balance_sheet[Intangible Assets]               : %s millions' % \
#           (balance_sheet['Intangible Assets'] / ONE_MILLION))

#     # print('balance_sheet[Deferred Long Term Asset Charges]: %s millions' % \
#     #       (balance_sheet['Deferred Long Term Asset Charges'] / ONE_MILLION))

    # check for "income"(s)
#     print('info[netIncomeToCommon]                           : %s millions' % \
#           (info['netIncomeToCommon'] / ONE_MILLION))
#     print('financials[Net Income]                            : %s millions' % \
#           (financials['Net Income'] / ONE_MILLION))
#     print('financials[Net Income From Continuing Ops]        : %s millions' % \
#           (financials['Net Income From Continuing Ops'] / ONE_MILLION))
#     print('financials[Net Income Applicable To Common Shares]: %s millions' % \
#           (financials['Net Income Applicable To Common Shares'] / ONE_MILLION))
    
#     print('financials[Income Before Tax]                     : %s millions' % \
#           (financials['Income Before Tax'] / ONE_MILLION))
#     print('financials[Operating Income]                      : %s millions' % \
#           (financials['Operating Income'] / ONE_MILLION))
#     print('financials[Income Tax Expense]                    : %s millions' % \
#           (financials['Income Tax Expense'] / ONE_MILLION))
#     print('financials[Total Other Income Expense Net]        : %s millions' % \
#           (financials['Total Other Income Expense Net'] / ONE_MILLION))

    # check for "cash"(s)
#     print('cash_flow[Total Cash From Financing Activities]     : %s millions' % \
#           (cash_flow['Total Cash From Financing Activities'] / ONE_MILLION))
#     print('cash_flow[Total Cash From Operating Activities]     : %s millions' % \
#           (cash_flow['Total Cash From Operating Activities'] / ONE_MILLION))
    
#     print('cash_flow[Total Cashflows From Investing Activities]: %s millions' % \
#           (cash_flow['Total Cashflows From Investing Activities'] / ONE_MILLION))
    
#     print('cash_flow[Other Cashflows From Financing Activities]: %s millions' % \
#           (cash_flow['Other Cashflows From Financing Activities'] / ONE_MILLION))
#     print('cash_flow[Other Cashflows From Investing Activities]: %s millions' % \
#           (cash_flow['Other Cashflows From Investing Activities'] / ONE_MILLION))
    
#     print('cash_flow[Change In Cash]                           : %s millions' % \
#           (cash_flow['Change In Cash'] / ONE_MILLION))

    # check for "dividend"(s)
#     print('info[trailingAnnualDividendYield]: %s' % (info['trailingAnnualDividendYield']))
#     print('info[trailingAnnualDividendRate] : %s' % (info['trailingAnnualDividendRate']))
    
#     print('info[dividendYield]              : %s' % (info['dividendYield']))
#     print('info[dividendRate]               : %s' % (info['dividendRate']))
    
#     print('info[lastDividendValue]          : %s' % (info['lastDividendValue']))
#     print('info[lastDividendDate]           : %s' % (info['lastDividendDate']))
    
#     print('info[exDividendDate]             : %s' % (info['exDividendDate']))
#     print('info[fiveYearAvgDividendYield]   : %s' % (info['fiveYearAvgDividendYield']))
    
    # ----- store necessary fields -----
    
    # from 'info'
    short_name = info['shortName']
    enterprise_value = info['enterpriseValue']
    market_cap = info['marketCap']
    trailing_annual_dividend_yield = info['trailingAnnualDividendYield']  # or 'Forward annual dividend yield'
    
    # from 'financials'
    ebit = financials['Ebit']
    net_income_applicable_to_common_shares = \
        financials['Net Income Applicable To Common Shares']  # or 'Net income available to common shareholders'
    
    # from balance_sheet
    total_current_assets = balance_sheet['Total Current Assets']
    total_current_liabilities = balance_sheet['Total Current Liabilities']
    property_plant_equipment = balance_sheet['Property Plant Equipment']  # or 'Net property, plant and equipment'
    total_stockholder_equity = balance_sheet['Total Stockholder Equity']
    long_term_debt = balance_sheet.get('Long Term Debt', None)  # not existing for DASH
    
    # from cash_flow
    depreciation = cash_flow['Depreciation']  # or 'Depreciation & amortisation'
    total_cash_from_operating_activities = \
        cash_flow['Total Cash From Operating Activities']  # or 'Net cash provided by operating activities'
    capital_expenditures = cash_flow.get('Capital Expenditures', None)
    
    # add the values to a dataframe
    df_dict = {
        'Symbol': ticker, 
               
        'Name': short_name,
        'Enterprise Value': enterprise_value,
        'Market Cap': market_cap,
        'Trailing Annual Dividend Yield': trailing_annual_dividend_yield,
        
        'EBIT': ebit, 
        'Net Income Applicable To Common Shares': net_income_applicable_to_common_shares,
        
        'Total Current Assets': total_current_assets, 
        'Total Current Liabilities': total_current_liabilities, 
        'Property Plant Equipment': property_plant_equipment,
        'Total Stockholder Equity': total_stockholder_equity,
        'Long Term Debt': long_term_debt,
        
        'Depreciation': depreciation,
        'Total Cash From Operating Activities': total_cash_from_operating_activities,
        'Capital Expenditures': capital_expenditures
    }    
    df = pd.DataFrame(df_dict, index=['1'])
    
    df_full = df_full.append(df, ignore_index=True)
    
print('--- Total time: %.2f seconds' % (time.time() - start_time_all))

Getting data for ABBV... Done in 2.24 seconds!
Getting data for ABT... Done in 3.79 seconds!
Getting data for AMT... Done in 2.85 seconds!
Getting data for AXP... Done in 2.35 seconds!
Getting data for BA... Done in 3.71 seconds!
Getting data for BAC... Done in 3.81 seconds!
Getting data for BLK... Done in 2.95 seconds!
Getting data for BMY... Done in 2.21 seconds!
Getting data for C... Done in 3.40 seconds!
Getting data for CAT... Done in 4.39 seconds!
--- Total time: 31.78 seconds


In [46]:
# ----------- store dataframe to a CSV file

start_time = time.time()
print('Saving data to %s... ' % EXPORT_TO_CSV_FILE, end="")

df_full.to_csv(EXPORT_TO_CSV_FILE)

print('Done in %.2f seconds!' % (time.time() - start_time))  

Saving data to data.csv... Done in 0.00 seconds!
